<a href="https://colab.research.google.com/github/zebaayaz/Regession-Model-using-1-D-Conventional-Neural-Network-/blob/master/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import the pandas library to read our dataset
import pandas as pd
#To split the data into training and test data
from sklearn.model_selection import train_test_split

#To manipulate data
import numpy as np
from keras.utils import to_categorical



Using TensorFlow backend.


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("gdrive/My Drive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:

# import data from the link
data = pd.read_csv('https://raw.githubusercontent.com/cacoderquan/Sentiment-Analysis-on-the-Rotten-Tomatoes-movie-review-dataset/master/train.tsv', sep='\t')
data.head(10)



,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2
5,6,1,of escapades demonstrating the adage that what...,2
6,7,1,of,2
7,8,1,escapades demonstrating the adage that what is...,2
8,9,1,escapades,2
9,10,1,demonstrating the adage that what is good for ...,2


In [4]:
# check the shape of df
data.shape

(156060, 4)

In [5]:
#libraries used for data preprocessing
import nltk
import random
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [6]:
#splitting each review into documents
documents = []

for i in range(data.shape[0]):
  tmpWords = word_tokenize(data['Phrase'][i])
  documents.append((tmpWords, data['Sentiment'][i]))

print(documents[1])

(['A', 'series', 'of', 'escapades', 'demonstrating', 'the', 'adage', 'that', 'what', 'is', 'good', 'for', 'the', 'goose'], 2)


In [7]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer, LancasterStemmer

porter = PorterStemmer()
lancaster=LancasterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()
stopwords_en = stopwords.words("english")
punctuations="?:!.,;'\"-()"

#parameters to adjust to see the impact on outcome
remove_stopwords = True
useStemming = True
useLemma = False
removePuncs = True

for l in range(len(documents)):
  label = documents[l][1]
  tmpReview = []
  for w in documents[l][0]:
    newWord = w
    if remove_stopwords and (w in stopwords_en):
      continue
    if removePuncs and (w in punctuations):
      continue
    if useStemming:
      
      newWord = lancaster.stem(newWord)
    if useLemma:
      newWord = wordnet_lemmatizer.lemmatize(newWord)
    tmpReview.append(newWord)
  documents[l] = (' '.join(tmpReview), label)
print(documents[1])

('a sery escapad demonst ad good goos', 2)


In [0]:
all_data = pd.DataFrame(documents, columns=['Phrase', 'Sentiment'])
x_train, x_test, y_train, y_test = train_test_split(all_data['Phrase'], all_data['Sentiment'], train_size = 0.7, shuffle = True, random_state = 2003)

In [0]:
#vectorize the data 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words = 'english', ngram_range= (1,1), max_features=1500)
X = vectorizer.fit_transform(all_data['Phrase'])
x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

In [0]:
x_train_np = x_train.toarray()
y_train_np = to_categorical(y_train)
x_test_np = x_test.toarray()
y_test_np = to_categorical(y_test)

In [0]:
x_train = np.expand_dims(x_train_np, axis=2)
x_test = np.expand_dims(x_test_np, axis=2)

In [0]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D

batch_size = 128

#model creation
model = Sequential()
#defing the layers
model.add(Conv1D(filters = 128, kernel_size=1, activation='relu', input_shape=(x_train_np.shape[1],1)))
model.add(Conv1D(filters = 128, kernel_size=1, activation='relu'))
model.add(MaxPooling1D(pool_size =2))
model.add(Flatten())
keras.layers.Dropout(0.1, noise_shape=None, seed=None)

model.add(Dense(5, activation='softmax'))

In [15]:
#model compilation and optimizer used is Stochastic Gradient Descent
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc'])

In [16]:
#Trainig the model
model.fit(x_train, y_train_np, validation_data=(x_test, y_test_np), epochs = 50, batch_size = 128)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 109242 samples, validate on 46818 samples
Epoch 1/50





109242/109242 [==============================] - 31s 288us/step - loss: 1.2926 - acc: 0.5082 - val_loss: 1.2803 - val_acc: 0.5123
Epoch 2/50
109242/109242 [==============================] - 18s 168us/step - loss: 1.2797 - acc: 0.5089 - val_loss: 1.2733 - val_acc: 0.5123
Epoch 3/50
109242/109242 [==============================] - 18s 169us/step - loss: 1.2717 - acc: 0.5089 - val_loss: 1.2623 - val_acc: 0.5125
Epoch 4/50
109242/109242 [==============================] - 18s 168us/step - loss: 1.2579 - acc: 0.5097 - val_loss: 1.2456 - val_acc: 0.5151
Epoch 5/50
109242/109242 [==============================] - 18s 168us/step - loss: 1.2357 - acc: 0.5143 - val_loss: 1.2209 - val_acc: 0.5250
Epoch 6/50
109242/109242 [==============================] - 18s 167us/step - loss: 1.2075 - acc: 0.5231 - val_loss: 1.1912 - val_acc: 0.5289
E

In [18]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
y_pred = model.predict_classes(x_test, batch_size=128, verbose=0)
y_pred1=np.argmax(y_test_np, axis=1)

accuracy = accuracy_score(y_pred1, y_pred)
precision = precision_score(y_pred1, y_pred, average='weighted')
f1 = f1_score(y_pred1, y_pred, average='weighted')
recall = recall_score(y_pred1, y_pred, average='weighted')
print('Accuracy: %f' % accuracy)
print('Precision: %f' % precision)
print('Recall: %f' % recall)
print('F1 score: %f' % f1)

Accuracy: 0.573583
Precision: 0.535278
Recall: 0.573583
F1 score: 0.522006


In [0]:
model.save('0888759_1dConv_reg.h5')

In [0]:
from keras.models import load_model
model = load_model('0888759_1dConv_reg.h5')